In [1]:
import open3d as o3d
import numpy as np
import cv2

# Load the depth map (as a grayscale image)
depth_image_path = "/Users/cmazzoleni/Downloads/visualization_result.png"
depth_image = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)  # Read the image
depth_image = depth_image.astype(np.float32)

# Filter out zero values (invalid depth) before normalization
valid_depth_mask = depth_image > 0

# Get minimum and maximum depth values, excluding zero
min_depth = np.min(depth_image[valid_depth_mask])
max_depth = np.max(depth_image[valid_depth_mask])

# Normalize the depth values between 0 and 1, excluding zero values
depth_image[valid_depth_mask] = (depth_image[valid_depth_mask] - min_depth) / (max_depth - min_depth)

# Create a mask to eliminate values that are either 0 or 1
valid_depth_mask = (depth_image > 0) & (depth_image < 1)

# Apply the mask (set the values that don't satisfy the condition to NaN)
depth_image[~valid_depth_mask] = np.nan

# Convert depth image to Open3D Image object
depth_o3d = o3d.geometry.Image(depth_image)
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(width=depth_image.shape[1], 
                         height=depth_image.shape[0], 
                         fx=692.8203230275511,  # Focal length in x
                         fy=692.8203230275511,  # Focal length in y
                         cx=depth_image.shape[1] / 2.0,  # Principal point in x
                         cy=depth_image.shape[0] / 2.0)  # Principal point in y

print(intrinsic.intrinsic_matrix)
#692.8203230275511, 0.0, 399.5
# Create a point cloud from the depth map
pcd = o3d.geometry.PointCloud.create_from_depth_image(depth_o3d, intrinsic)

# Optionally, remove points with invalid depth (NaN or infinite values)
pcd = pcd.remove_non_finite_points()

mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
#center the mesh
mesh.translate(-mesh.get_center())
#set_camera_position(visualizer, center=np.array([0, 0, 0]), distance=fixed_distance, view_name="right")

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd, mesh], window_name="Reconstructed Point Cloud")



[[692.82032303   0.         400.        ]
 [  0.         692.82032303 400.        ]
 [  0.           0.           1.        ]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


: 

In [1]:
#path setup
import os
import sys
# Define the path to the project root directory (one level above `src`)
project_root_path = "/Users/cmazzoleni/Documents/GitHub/CuboidAbstractionViaSeg"
# Add the src directory to sys.path
sys.path.append(os.path.join(project_root_path))
import numpy as np
import open3d as o3d
import argparse
from src.utils.dataset_utils import create_directory
from src.visualizers.Open3dVisualizer import Open3DVisualizer
from src.utils.json_utils import load_cuboid_data
from src.utils.plotting_utils import load_mesh_data, load_pointcloud_data
import pprint

In [15]:
def set_camera_position(visualizer, center, distance, view_name="front"):
    ctr = visualizer.vis.get_view_control()

    # Camera position will be distance units away from the center along the z-axis
    camera_position = center + np.array([0, 0, distance])

    # Set the camera parameters
    ctr.set_lookat(camera_position.tolist())  # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name])  # Set the view direction)  # Camera looking straight down the z-axis
    
    # Adjust zoom if necessary
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])


    # Apply the camera extrinsics
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    extrinsic = { 
        "left": np.array([
    [0, 0, 1, 0],    # Rotate to look along the positive x-axis
    [0, 1, 0, 0],    # No rotation around y
    [-1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "right": np.array([
    [0, 0, -1, 0],   # Rotate to look along the negative x-axis
    [0, 1, 0, 0],    # No rotation around y
    [1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "top": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, 1, 0],    # Rotate to look along the positive z-axis
    [0, -1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "bottom": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, -1, 0],   # Rotate to look along the negative z-axis
    [0, 1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "front": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, 1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ]),
        "back": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, -1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ])
    }
    
    camera_params.extrinsic = extrinsic[view_name]
    ctr.convert_from_pinhole_camera_parameters(camera_params, allow_arbitrary=True)
     # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name]) # Camera looking straight down the z-axis
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])

    visualizer.vis.poll_events()
    visualizer.vis.update_renderer()
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    print(camera_params.extrinsic)

In [16]:
views = {
            "top": [0, 1, 0],
            "bottom": [0, -1, 0],
            "left": [1, 0, 0],
            "right": [-1, 0, 0],
            "front": [0, 0, -1],
            "back": [0, 0, 1]
        }

colors = {
    "top": [1, 0, 0],      # Red
    "bottom": [0, 1, 0],   # Green
    "left": [0, 0, 1],     # Blue
    "right": [1, 1, 0],    # Yellow
    "front": [1, 0, 1],    # Magenta
    "back": [0, 1, 1]      # Cyan
}

In [24]:
import open3d as o3d

visualizer = Open3DVisualizer()
mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
#center the mesh
mesh.translate(-mesh.get_center())
visualizer.add_geometry(mesh)
depth_image_path = "/Users/cmazzoleni/Downloads/visualization_result.png"
depth_image = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)  # Read the image
depth_image = depth_image.astype(np.float32)

# Filter out zero values (invalid depth) before normalization
valid_depth_mask = depth_image > 0

# Get minimum and maximum depth values, excluding zero
min_depth = np.min(depth_image[valid_depth_mask])
max_depth = np.max(depth_image[valid_depth_mask])

# Normalize the depth values between 0 and 1, excluding zero values
depth_image[valid_depth_mask] = (depth_image[valid_depth_mask] - min_depth) / (max_depth - min_depth)

# Create a mask to eliminate values that are either 0 or 1
valid_depth_mask = (depth_image > 0) & (depth_image < 1)

# Apply the mask (set the values that don't satisfy the condition to NaN)
depth_image[~valid_depth_mask] = np.nan

# Convert depth image to Open3D Image object
depth_o3d = o3d.geometry.Image(depth_image)
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(width=depth_image.shape[1], 
                         height=depth_image.shape[0], 
                         fx=692.8203230275511,  # Focal length in x
                         fy=692.8203230275511,  # Focal length in y
                         cx=depth_image.shape[1] / 2.0,  # Principal point in x
                         cy=depth_image.shape[0] / 2.0)  # Principal point in y

print(intrinsic.intrinsic_matrix)
#692.8203230275511, 0.0, 399.5
# Create a point cloud from the depth map
pcd = o3d.geometry.PointCloud.create_from_depth_image(depth_o3d, intrinsic)
#extract points
pcd_points = pcd.points
pcd_colors = pcd.colors
visualizer.add_pointcloud("pcd", pcd_points, pcd_colors)

fixed_distance = 1.0
#set_camera_position(visualizer, center=np.array([0, 0, 0]), distance=fixed_distance, view_name="front")
visualizer.show()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[[692.82032303   0.         400.        ]
 [  0.         692.82032303 400.        ]
 [  0.           0.           1.        ]]


In [19]:
import open3d as o3d
import cv2
import numpy as np

# Load depth image as numpy array and normalize
depth_image_path = "/Users/cmazzoleni/Downloads/visualization_result.png"

depth_image_path = "mesh_depth_image.png"
depth_image = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)  # Read the image
depth_image = depth_image.astype(np.float32)


# Filter out zero values (invalid depth) before normalization
valid_depth_mask = depth_image > 0

# Get minimum and maximum depth values, excluding zero
min_depth = np.min(depth_image[valid_depth_mask])
max_depth = np.max(depth_image[valid_depth_mask])

# Normalize the depth values between 0 and 1, excluding zero values
depth_image[valid_depth_mask] = (depth_image[valid_depth_mask] - min_depth) / (max_depth - min_depth)

# Create a mask to eliminate values that are either 0 or 1
valid_depth_mask = (depth_image > 0) & (depth_image < 1)

# Apply the mask (set the values that don't satisfy the condition to NaN)
depth_image[~valid_depth_mask] = np.nan

# Convert depth image to Open3D Image object
depth_o3d = o3d.geometry.Image(depth_image)

# Intrinsics setup
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(width=depth_image.shape[1], 
                         height=depth_image.shape[0], 
                         fx=692.8203230275511,  # Focal length in x
                         fy=692.8203230275511,  # Focal length in y
                         cx=depth_image.shape[1] / 2.0,  # Principal point in x
                         cy=depth_image.shape[0] / 2.0)  # Principal point in y


# Create a point cloud from the depth map using both intrinsic and extrinsic parameters
pcd = o3d.geometry.PointCloud.create_from_depth_image(
    depth_o3d, 
    intrinsic,
)


# After creating the point cloud
# Apply a transformation to flip the point cloud along the Z-axis
flip_transform = np.array([[1, 0, 0, 0], 
                           [0, -1, 0, 0], 
                           [0, 0, 1, 0],  
                           [0, 0, 0, 1]])

# Transform the point cloud
pcd.transform(flip_transform)

mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
#center the mesh
mesh.translate(-mesh.get_center())


# Visualize the point cloud again
o3d.visualization.draw_geometries([pcd, mesh])


In [4]:
def set_camera_position(visualizer, center, distance, view_name="front"):
    ctr = visualizer.get_view_control()

    # Camera position will be distance units away from the center along the z-axis
    camera_position = center + np.array([0, 0, distance])

    # Set the camera parameters
    ctr.set_lookat(camera_position.tolist())  # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name])  # Set the view direction)  # Camera looking straight down the z-axis
    
    # Adjust zoom if necessary
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])


    # Apply the camera extrinsics
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    extrinsic = { 
        "left": np.array([
    [0, 0, 1, 0],    # Rotate to look along the positive x-axis
    [0, 1, 0, 0],    # No rotation around y
    [-1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "right": np.array([
    [0, 0, -1, 0],   # Rotate to look along the negative x-axis
    [0, 1, 0, 0],    # No rotation around y
    [1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "top": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, 1, 0],    # Rotate to look along the positive z-axis
    [0, -1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "bottom": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, -1, 0],   # Rotate to look along the negative z-axis
    [0, 1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "front": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, 1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ]),
        "back": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, -1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ])
    }
    
    camera_params.extrinsic = extrinsic[view_name]
    ctr.convert_from_pinhole_camera_parameters(camera_params, allow_arbitrary=True)
     # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name]) # Camera looking straight down the z-axis
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])

    visualizer.poll_events()
    visualizer.update_renderer()
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    return camera_params

In [2]:
def capture_front_depth(visualizer, output_filename: str, distance: float = 1.0):
    """
    Captures and saves a screenshot of the scene from the front view.

    Parameters:
    - output_filename: The path where the image will be saved.
    - distance: The distance of the camera from the center of the scene.
    """
    ctr = visualizer.vis.get_view_control()
    
    # Set the front view (camera looking along the negative Z-axis)
    ctr.set_front([0, 0, -1])  # Camera looking towards the negative z-axis
    ctr.set_lookat([0, 0, 0])  # Center of the scene
    ctr.set_up([0, 1, 0])      # The up direction is along the Y-axis

    # Apply the camera extrinsics for front view
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    extrinsic_front = np.array([
        [1, 0, 0, 0],    # No rotation around X
        [0, 1, 0, 0],    # No rotation around Y
        [0, 0, 1, distance],  # Camera positioned at Z = distance
        [0, 0, 0, 1]
    ])
    camera_params.extrinsic = extrinsic_front
    ctr.convert_from_pinhole_camera_parameters(camera_params, allow_arbitrary=True)

    # Poll events and update the renderer to apply changes
    visualizer.vis.poll_events()
    visualizer.vis.update_renderer()

    try:
        # Capture and save the screenshot
        visualizer.vis.capture_depth_image(output_filename)
        print(f"Front view screenshot saved as {output_filename}")
    except Exception as e:
        print(f"Failed to save front view screenshot: {e}")

In [15]:
import open3d as o3d
import cv2
import numpy as np

#depth_image_path = "depth_image.png"
depth_image_path = "/Users/cmazzoleni/Downloads/visualization_result.png"
#depth_image_path = "/Users/cmazzoleni/Downloads/visualization_result.png"
depth_image = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)  # Read the image
depth_image = depth_image.astype(np.float32)

min_depth = np.min(depth_image)
max_depth = np.max(depth_image)
# Normalize the depth values between 0 and 1, excluding zero values
depth_image = (depth_image - min_depth) / (max_depth - min_depth)

# Create a mask to eliminate values that are either 0 or 1
valid_depth_mask = (depth_image > 0) & (depth_image < 1)

# Apply the mask (set the values that don't satisfy the condition to NaN)
depth_image[~valid_depth_mask] = np.nan

# Convert depth image to Open3D Image object
depth_o3d = o3d.geometry.Image(depth_image)

print(depth_image.shape[1])
print(depth_image.shape[0])

# Intrinsics setup
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(width=depth_image.shape[1], 
                         height=depth_image.shape[0], 
                         fx=692.8203230275511,  # Focal length in x
                         fy=692.8203230275511,  # Focal length in y
                         cx=depth_image.shape[1] / 2.0,  # Principal point in x
                         cy=depth_image.shape[0] / 2.0)  # Principal point in y

extrinsic_front = np.array([
        [1, 0, 0, 0],    # No rotation around X
        [0, 1, 0, 0],    # No rotation around Y
        [0, 0, -1, 1],  # Camera positioned at Z = distance
        [0, 0, 0, 1]
    ])
  
# Create a point cloud from the depth map using both intrinsic and extrinsic parameters
pcd = o3d.geometry.PointCloud.create_from_depth_image(
    depth_o3d, 
    intrinsic,
    extrinsic= extrinsic_front
)

flip_transform = np.array([[1, 0, 0, 0], 
                           [0, -1, 0, 0], 
                           [0, 0, 1, 0],  
                           [0, 0, 0, 1]])

# Transform the point cloud
pcd.transform(flip_transform)


mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
#center the mesh
mesh.translate(-mesh.get_center())
# Calculate the maximum distance from the center

# Visualize the point cloud and the scaled mesh
o3d.visualization.draw_geometries([pcd, mesh])

800
800


: 

In [5]:
visualizer = Open3DVisualizer()

#load mesh
mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
#center the mesh
mesh.translate(-mesh.get_center())
set_camera_position(visualizer, np.array([0,0,0]), 1)
visualizer.add_geometry(mesh)
#take the depth map from front

capture_front_depth(visualizer, "mesh_depth_image.png")
visualizer.show()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


AttributeError: 'Open3DVisualizer' object has no attribute 'get_view_control'